## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
#from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all

from functions.aux import MetricTotalAccuraccy
# Load tree transformation
from functions.transform_inference import TransformerInference

from net import NeoNet

class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


IS CUDA AVAILABLE: True


131072

### Basic parameters

In [2]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.2_subqueries'
model_name = 'Model_Adam_0.000015'
model_path = "./" + model_name + '.pt'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"
optimizer = "Adam"
#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
#min_time_or = 15
#max_time_or = 80
#percent_of_data = 1
#min_time = 5
#max_time = 80


In [3]:
epsilon = 0.75 ## EPSILON
lr_rl = 0.09 ## LEARNING RATE RL
disc_fac = 0.999 ## DISCOUNT FACTOR RL 
epoch_rl = 200 ### TOTAL EPOCH POR DATA RL
n = model_name + str(lr_rl)##SALT Nombre archivos

### Features

In [4]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

In [5]:
ds_train = pd.read_csv(URL + csv_name + '_ds_train_ft.csv', engine='python', encoding='utf-8')
ds_val = pd.read_csv(URL + csv_name + '_ds_val_ft.csv', engine='python', encoding='utf-8')
ds_test = pd.read_csv(URL + csv_name + '_ds_test_ft.csv', engine='python', encoding='utf-8')
ds_rl = pd.read_csv(URL + csv_name + '_ds_rl_ft.csv', engine='python', encoding='utf-8')

In [6]:
print("ds_train.shape",ds_train.shape)
print("ds_val.shape",ds_val.shape)
print("ds_test.shape",ds_test.shape)
print("ds_rl.shape",ds_rl.shape)

ds_train.shape (6849, 15)
ds_val.shape (1476, 15)
ds_test.shape (1479, 15)
ds_rl.shape (1845, 67)


# FUNCTIONS

In [7]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [8]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

Size Train: 6849, Val 1476


In [9]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [10]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (6849, 15)
shape ds_val  : (1476, 15)
shape ds_test : (1479, 15)
shape ds_rl : (1845, 67)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (1476, 5)
shape x_train_query: (6849, 5)
----------x_plan_level_data----------
shape x_val_tree  : (1476,)
shape x_train_tree: (6849,)
----------y_data------------
shape y_val  : (1476,)
shape y_train: (6849,)

----------TEST DATA----------
shape x_test_tree : (1479,)
shape x_test_query: (1479, 5)
shape y_test      : (1479,)
-----------------------

----------RL DATA----------
shape x_rl_tree : (1845,)
shape x_rl_query: (1845, 5)
shape y_rl      : (1845,)
-----------------------


In [11]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

0

In [12]:
data_inference = {}
bao_base = ""
with (open(bao_base + "preprocess_inference_info_" + model_name + ".pickle", "rb")) as openfile:
    while True:
        try:
            data_inference = pickle.load(openfile)
        except EOFError:
            break
# Load Scaler for query features.
scalerx = None
with (open(bao_base + "scalerx_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            scalerx = pickle.load(openfile)
        except EOFError:
            break

pipeline_inverse = None
with (open(bao_base + "pipeline_inverse_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            pipeline_inverse = pickle.load(openfile)
        except EOFError:
            break
with open('NeoNetValues_' + model_name +'.txt', 'r') as file:
    data = file.read().rstrip()
neo_data = ast.literal_eval(data)

IS CUDA AVAILABLE: True


In [13]:
def ActivationFuncReluOrLeaky(string):
    if string == "<class 'torch.nn.modules.activation.LeakyReLU'>":
        return nn.LeakyReLU
    elif string == "<class 'torch.nn.modules.activation.ReLU'>":
        return nn.ReLU
io_dim_model_data = int(neo_data['io_dim'])
query_input_size = int(neo_data['query_input_size'])
query_hidden_inputs = ast.literal_eval(neo_data['query_hidden_inputs'])
query_output = int(neo_data['query_output'])
tree_units = ast.literal_eval(neo_data['tree_units'])
tree_units_dense = ast.literal_eval(neo_data['tree_units_dense'])
tree_activation_tree = ActivationFuncReluOrLeaky(neo_data['activation_tree'])
tree_activation_dense = ActivationFuncReluOrLeaky(neo_data['activation_dense'])

def get_prediction_dataset_from_load(ds, list_columns):
    x_tree = ds["trees"].values
    x_query = ds[list_columns]
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    targets = list(ds["time"].values)
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}

def get_prediction_from_load(x_tree, x_query, targets):
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}


In [14]:
reg = NeoNet(
    io_dim_model_data,
    query_input_size,
    query_hidden_inputs,
    query_output=query_output,
    tree_units=tree_units,
    tree_units_dense=tree_units_dense,
    activation_tree=tree_activation_tree,
    activation_dense=tree_activation_dense,
)
device = torch.device("cuda")
# Loading model parameters.
reg_path = model_path
#reg.load_state_dict(torch.load(reg_path, map_location=torch.device('cpu')))
reg.load_state_dict(torch.load(reg_path))
reg.to(device)
transformer = TransformerInference(
        model=reg,
        tree_transform=data_inference["tree_transform"],
        pipeline_inverse=pipeline_inverse,
        scalerx=scalerx,
        maxcardinality=data_inference["maxcardinality"]
    )
print("Modelos Listos")

Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(1282, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias=True)]
Modelos Listos


In [15]:
 ds_test['time']

0         1.112900
1         1.983621
2         1.503600
3         1.404750
4         1.669234
           ...    
1474    102.446694
1475    148.000000
1476    107.995152
1477    113.000000
1478    102.963054
Name: time, Length: 1479, dtype: float64

In [16]:
results = get_prediction_from_load(ds_test['trees'].values, ds_test[list_columns], ds_test['time'])

In [17]:
results

{'pred': [2.989485263824463,
  2.218127727508545,
  17.620981216430664,
  98.83006286621094,
  27.08061981201172,
  17.293182373046875,
  16.92856788635254,
  2.8454933166503906,
  18.2322940826416,
  18.086599349975586,
  27.161592483520508,
  2.7694365978240967,
  17.955974578857422,
  20.808757781982422,
  19.195518493652344,
  2.967419147491455,
  18.014310836791992,
  3.139894485473633,
  3.920468807220459,
  24.535791397094727,
  17.718395233154297,
  18.417705535888672,
  28.99654769897461,
  3.918849468231201,
  19.878250122070312,
  20.047168731689453,
  69.48811340332031,
  3.015251636505127,
  20.35631561279297,
  59.29173278808594,
  4.084952354431152,
  62.46084213256836,
  5.259437561035156,
  19.56340980529785,
  2.6011180877685547,
  3.591683864593506,
  7.089659690856934,
  3.1300439834594727,
  18.56989860534668,
  40.47145462036133,
  2.9810664653778076,
  103.60863494873047,
  14.421701431274414,
  17.68401527404785,
  16.563535690307617,
  17.102659225463867,
  3.3

## Reinforcement Learning

In [18]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    #tree_format = TreeFormat_all(new_dicto,symbol)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
    reward -= pred['pred'][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    #new_tree = np.array([str(TreeFormat_all(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
        reward -= pred['pred'][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        if q_val[act[0]] == 0:
            q_val[act[0]] = reward
        else:
            q_val[act[0]] = max(reward,q_val[act[0]])
    return q_value_neo

def RL_max_available_actions(q_value, old_states, actions):
    available_actions = RL_available_actions(actions, old_states)
    index_av = [i[0] for i in available_actions]
    q_value_acotado = q_value[index_av[0]:index_av[-1]+1]
    return np.max(q_value_acotado)

In [19]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

----------RL DATA----------
shape ds_rl : (1845, 67)
shape x_rl_tree : (1845,)
shape x_rl_query: (1845, 5)
shape y_rl      : (1845,)
-----------------------


## MULTI-VAL

##### Seleccionar data

In [20]:
min_num_actions = -1
buenos_idx = []
aaa = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        aaa.append(y_rl_ind)
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_idx",total_buenos_idx)
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree',
            #'history_old',
            #'history_new'
             ]
 
no_history = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree'
             ]

total_idx 1845


##### Parameters

In [21]:
#epsilon = 0.7 ## EPSILON
#lr_rl = 0.9 ## LEARNING RATE
#disc_fac = 0.999 ## DISCOUNT FACTOR
#epoch_rl = 200 ### TOTAL EPOCH POR DATA
#n = model_name ##SALT Nombre archivos

### Q - Learning - multival

##### Parameters

In [22]:
epsilon_ql = epsilon ## EPSILON
lr_ql = lr_rl ## LEARNING RATE
disc_fac_ql = disc_fac ## DISCOUNT FACTOR
epoch_ql = epoch_rl ### TOTAL EPOCH POR DATA

##### Loop

In [23]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)
            
            new_state = len(current_state)-1
            old_q_value = q_value_ql[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_ql[new_state,:], old_states_val, actions)
            temporal_difference = reward + disc_fac_ql * q_max - old_q_value
            #temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

Start: 2022-01-01 02:10:38.625650
2022-01-01 02:11:18.784465 : 10/1845, index 9 evaluating
2022-01-01 02:12:06.075784 : 20/1845, index 19 evaluating
2022-01-01 02:12:58.981907 : 30/1845, index 29 evaluating
2022-01-01 02:13:45.064762 : 40/1845, index 39 evaluating
2022-01-01 02:14:35.373131 : 50/1845, index 49 evaluating
2022-01-01 02:15:25.697194 : 60/1845, index 59 evaluating
2022-01-01 02:16:18.955561 : 70/1845, index 69 evaluating
2022-01-01 02:17:10.778789 : 80/1845, index 79 evaluating
2022-01-01 02:18:06.778877 : 90/1845, index 89 evaluating
2022-01-01 02:19:00.011314 : 100/1845, index 99 evaluating
2022-01-01 02:20:10.023606 : 110/1845, index 109 evaluating
2022-01-01 02:20:59.608011 : 120/1845, index 119 evaluating
2022-01-01 02:21:46.011648 : 130/1845, index 129 evaluating
2022-01-01 02:22:36.491944 : 140/1845, index 139 evaluating
2022-01-01 02:23:27.025392 : 150/1845, index 149 evaluating
2022-01-01 02:24:20.494857 : 160/1845, index 159 evaluating
2022-01-01 02:25:11.219425

2022-01-01 04:15:30.675592 : 1370/1845, index 1369 evaluating
2022-01-01 04:16:30.994997 : 1380/1845, index 1379 evaluating
2022-01-01 04:17:27.178220 : 1390/1845, index 1389 evaluating
2022-01-01 04:18:23.210359 : 1400/1845, index 1399 evaluating
2022-01-01 04:19:16.468056 : 1410/1845, index 1409 evaluating
2022-01-01 04:20:09.668895 : 1420/1845, index 1419 evaluating
2022-01-01 04:21:06.340104 : 1430/1845, index 1429 evaluating
2022-01-01 04:22:00.848998 : 1440/1845, index 1439 evaluating
2022-01-01 04:22:57.261631 : 1450/1845, index 1449 evaluating
2022-01-01 04:23:54.977522 : 1460/1845, index 1459 evaluating
2022-01-01 04:24:53.981053 : 1470/1845, index 1469 evaluating
2022-01-01 04:25:51.762312 : 1480/1845, index 1479 evaluating
2022-01-01 04:26:45.073488 : 1490/1845, index 1489 evaluating
2022-01-01 04:27:49.593398 : 1500/1845, index 1499 evaluating
2022-01-01 04:28:49.646373 : 1510/1845, index 1509 evaluating
2022-01-01 04:29:41.681881 : 1520/1845, index 1519 evaluating
2022-01-

In [24]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_no_history = ql_df[no_history]
ql_df_no_history.to_csv(URL + 'rl_csvs/ql_df_no_history'+str(n)+'.csv',index=False)

In [25]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final'+str(n)+'.csv',index=False)

In [26]:
ql_df_no_history

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,50.027851,51.471825,50.0,50.0,0.000776,2.166268,0.027851,1.471825,0.027851,1.471825,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,0,1,51.476654,51.573895,50.0,50.0,2.180507,2.477144,1.476654,1.573895,1.476654,1.573895,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
2,0,2,51.134785,51.885372,50.0,50.0,1.287736,3.554628,1.134785,1.885372,1.134785,1.885372,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
3,0,3,51.808941,51.626541,50.0,50.0,3.272267,2.645636,1.808941,1.626541,1.808941,1.626541,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
4,0,4,49.252613,50.452660,50.0,50.0,0.558587,0.204901,0.747387,0.452660,0.747387,0.452660,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368995,1844,195,86.363327,76.015854,119.0,119.0,1065.152423,1847.636818,32.636673,42.984146,32.636673,42.984146,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368996,1844,196,88.976494,77.125496,119.0,119.0,901.410922,1753.474093,30.023506,41.874504,30.023506,41.874504,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368997,1844,197,87.521027,79.393845,119.0,119.0,990.925766,1568.647545,31.478973,39.606155,31.478973,39.606155,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368998,1844,198,86.149506,82.418022,119.0,119.0,1079.154981,1338.241103,32.850494,36.581978,32.850494,36.581978,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."


In [27]:
ql_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,52.054359,50.731041,50.0,50.0,4.220393,0.534421,2.054359,0.731041,2.054359,0.731041,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,1,199,43.631237,44.157562,50.0,50.0,40.561142,34.134079,6.368763,5.842438,6.368763,5.842438,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
2,2,199,49.748642,48.124676,50.0,50.0,0.063181,3.516841,0.251358,1.875324,0.251358,1.875324,"['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V..."
3,3,199,59.854527,59.187866,50.0,50.0,97.111693,84.416886,9.854527,9.187866,9.854527,9.187866,"['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire..."
4,4,199,42.145683,46.914783,50.0,50.0,61.690291,9.518561,7.854317,3.085217,7.854317,3.085217,"['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1840,199,77.586700,76.939972,109.0,109.0,986.795389,1027.845400,31.413300,32.060028,31.413300,32.060028,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1841,1841,199,57.954617,59.424068,130.0,130.0,5190.537277,4980.962114,72.045383,70.575932,72.045383,70.575932,"['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']"
1842,1842,199,77.044250,71.877586,119.0,119.0,1760.284917,2220.521867,41.955750,47.122414,41.955750,47.122414,"['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_VAR_V..."
1843,1843,199,81.486122,82.370819,118.0,118.0,1333.263277,1269.438532,36.513878,35.629181,36.513878,35.629181,"['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA..."


## Sarsa Multival

###### Parameters

In [34]:
epsilon_sar = epsilon ## EPSILON
lr_sar = lr_rl ## LEARNING RATE
disc_fac_sar = disc_fac ## DISCOUNT FACTOR
epoch_sar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [ ]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            new_state = len(current_state)-1
            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

In [36]:
sarsa_df = pd.DataFrame(values_sar,columns=rl_columns)
sarsa_df_no_history = sarsa_df[no_history]
sarsa_df_no_history.to_csv(URL + 'rl_csvs/sarsa_df_no_history'+str(n)+'.csv',index=False)

In [37]:
sarsa_df_final = pd.DataFrame(values_final_sar,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final'+str(n)+'.csv',index=False)

In [38]:
sarsa_df_no_history

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,52.740650,51.460743,50.0,50.0,7.511163,2.133770,2.740650,1.460743,2.740650,1.460743,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,0,1,50.231979,50.995544,50.0,50.0,0.053814,0.991109,0.231979,0.995544,0.231979,0.995544,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
2,0,2,50.405060,51.328804,50.0,50.0,0.164073,1.765720,0.405060,1.328804,0.405060,1.328804,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
3,0,3,51.486313,51.705799,50.0,50.0,2.209126,2.909751,1.486313,1.705799,1.486313,1.705799,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
4,0,4,50.416901,49.538551,50.0,50.0,0.173806,0.212935,0.416901,0.461449,0.416901,0.461449,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368995,1844,195,89.216820,78.897644,119.0,119.0,887.037825,1608.198953,29.783180,40.102356,29.783180,40.102356,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368996,1844,196,86.760689,74.717026,119.0,119.0,1039.373188,1960.981808,32.239311,44.282974,32.239311,44.282974,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368997,1844,197,85.214890,76.728416,119.0,119.0,1141.433690,1786.886776,33.785110,42.271584,33.785110,42.271584,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368998,1844,198,90.137047,82.381516,119.0,119.0,833.070067,1340.913407,28.862953,36.618484,28.862953,36.618484,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."


In [39]:
sarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,54.902344,48.818466,50.0,50.0,24.032974,1.396022,4.902344,1.181534,4.902344,1.181534,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,1,199,44.258114,43.385620,50.0,50.0,32.969256,43.750021,5.741886,6.614380,5.741886,6.614380,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
2,2,199,51.095856,55.166595,50.0,50.0,1.200900,26.693709,1.095856,5.166595,1.095856,5.166595,"['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V..."
3,3,199,58.679939,64.021523,50.0,50.0,75.341346,196.603094,8.679939,14.021523,8.679939,14.021523,"['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire..."
4,4,199,46.099068,45.989697,50.0,50.0,15.217273,16.082534,3.900932,4.010303,3.900932,4.010303,"['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1840,199,84.441147,73.630455,109.0,109.0,603.137268,1251.004712,24.558853,35.369545,24.558853,35.369545,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1841,1841,199,58.237816,57.377262,130.0,130.0,5149.811073,5274.062058,71.762184,72.622738,71.762184,72.622738,"['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']"
1842,1842,199,79.105896,76.963348,119.0,119.0,1591.539534,1767.080079,39.894104,42.036652,39.894104,42.036652,"['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_VAR_V..."
1843,1843,199,89.937363,82.258751,118.0,118.0,787.511614,1277.436886,28.062637,35.741249,28.062637,35.741249,"['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA..."


## Expected-Sarsa Multival

###### Parameters

In [40]:
epsilon_exsar = epsilon ## EPSILON
epsilon_exsar_alg = 0.9
lr_exsar = lr_rl ## LEARNING RATE
disc_fac_exsar = disc_fac ## DISCOUNT FACTOR
epoch_exsar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [41]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_exsar[new_state,:], old_states_val, actions)
            #q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar_alg/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar_alg)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

Start: 2022-01-01 09:56:14.343432
2022-01-01 09:56:57.929900 : 10/1845, index 9 evaluating
2022-01-01 09:57:47.472592 : 20/1845, index 19 evaluating
2022-01-01 09:58:42.795977 : 30/1845, index 29 evaluating
2022-01-01 09:59:31.227346 : 40/1845, index 39 evaluating
2022-01-01 10:00:23.764290 : 50/1845, index 49 evaluating
2022-01-01 10:01:15.738261 : 60/1845, index 59 evaluating
2022-01-01 10:02:11.286753 : 70/1845, index 69 evaluating
2022-01-01 10:03:04.568630 : 80/1845, index 79 evaluating
2022-01-01 10:04:03.057721 : 90/1845, index 89 evaluating
2022-01-01 10:04:58.337442 : 100/1845, index 99 evaluating
2022-01-01 10:06:10.661713 : 110/1845, index 109 evaluating
2022-01-01 10:07:01.783288 : 120/1845, index 119 evaluating
2022-01-01 10:07:50.000203 : 130/1845, index 129 evaluating
2022-01-01 10:08:42.732145 : 140/1845, index 139 evaluating
2022-01-01 10:09:35.297037 : 150/1845, index 149 evaluating
2022-01-01 10:10:31.197832 : 160/1845, index 159 evaluating
2022-01-01 10:11:23.943539

2022-01-01 12:04:36.086625 : 1370/1845, index 1369 evaluating
2022-01-01 12:05:37.668857 : 1380/1845, index 1379 evaluating
2022-01-01 12:06:35.191330 : 1390/1845, index 1389 evaluating
2022-01-01 12:07:32.470577 : 1400/1845, index 1399 evaluating
2022-01-01 12:08:26.915591 : 1410/1845, index 1409 evaluating
2022-01-01 12:09:21.955174 : 1420/1845, index 1419 evaluating
2022-01-01 12:10:19.920463 : 1430/1845, index 1429 evaluating
2022-01-01 12:11:16.097991 : 1440/1845, index 1439 evaluating
2022-01-01 12:12:13.084207 : 1450/1845, index 1449 evaluating
2022-01-01 12:13:11.735771 : 1460/1845, index 1459 evaluating
2022-01-01 12:14:12.179643 : 1470/1845, index 1469 evaluating
2022-01-01 12:15:10.663294 : 1480/1845, index 1479 evaluating
2022-01-01 12:16:04.748021 : 1490/1845, index 1489 evaluating
2022-01-01 12:17:10.963492 : 1500/1845, index 1499 evaluating
2022-01-01 12:18:12.460575 : 1510/1845, index 1509 evaluating
2022-01-01 12:19:05.580537 : 1520/1845, index 1519 evaluating
2022-01-

In [42]:
exsarsa_df = pd.DataFrame(values_exsar,columns=rl_columns)
exsarsa_df_no_history = exsarsa_df[no_history]
exsarsa_df_no_history.to_csv(URL + 'rl_csvs/exsarsa_df_no_history'+str(n)+'.csv',index=False)

In [43]:
exsarsa_df_final = pd.DataFrame(values_final_exsar,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final'+str(n)+'.csv',index=False)

In [44]:
exsarsa_df_no_history

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,50.677086,50.267498,50.0,50.0,0.458445,0.071555,0.677086,0.267498,0.677086,0.267498,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,0,1,50.986534,49.751488,50.0,50.0,0.973250,0.061758,0.986534,0.248512,0.986534,0.248512,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
2,0,2,51.476719,52.353466,50.0,50.0,2.180699,5.538802,1.476719,2.353466,1.476719,2.353466,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
3,0,3,50.866234,53.184177,50.0,50.0,0.750361,10.138986,0.866234,3.184177,0.866234,3.184177,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
4,0,4,51.397743,53.022888,50.0,50.0,1.953686,9.137853,1.397743,3.022888,1.397743,3.022888,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368995,1844,195,86.180679,77.631310,119.0,119.0,1077.107810,1711.368553,32.819321,41.368690,32.819321,41.368690,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368996,1844,196,87.886559,75.910233,119.0,119.0,968.046240,1856.728059,31.113441,43.089767,31.113441,43.089767,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368997,1844,197,85.374710,76.763229,119.0,119.0,1130.660122,1783.944793,33.625290,42.236771,33.625290,42.236771,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."
368998,1844,198,86.710739,75.864410,119.0,119.0,1042.596367,1860.679090,32.289261,43.135590,32.289261,43.135590,"['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P5044;"", [""VAR_VAR_V..."


In [45]:
exsarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,51.545437,51.675938,50.0,50.0,2.388375,2.808767,1.545437,1.675938,1.545437,1.675938,"['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P646;""]']"
1,1,199,42.971752,42.470367,50.0,50.0,49.396268,56.695367,7.028248,7.529633,7.028248,7.529633,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
2,2,199,48.535244,50.674931,50.0,50.0,2.145510,0.455531,1.464756,0.674931,1.464756,0.674931,"['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2045;"", [""VAR_URI_V..."
3,3,199,58.843967,56.066929,50.0,50.0,78.215760,36.807626,8.843967,6.066929,8.843967,6.066929,"['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P582;ᶲIRI_ID;...dire..."
4,4,199,47.644066,46.139973,50.0,50.0,5.550426,14.899811,2.355934,3.860027,2.355934,3.860027,"['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P176;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,1840,199,78.213074,72.074821,109.0,109.0,947.834829,1363.468809,30.786926,36.925179,30.786926,36.925179,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1841,1841,199,58.405415,58.396294,130.0,130.0,5125.784661,5127.090764,71.594585,71.603706,71.594585,71.603706,"['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']","['[""VAR_URI_VARᶲIRI_ID;...direct/P570;""]']"
1842,1842,199,86.115067,73.027893,119.0,119.0,1081.418849,2113.434616,32.884933,45.972107,32.884933,45.972107,"['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2386;"", [""VAR_VAR_V..."
1843,1843,199,84.349205,83.012047,118.0,118.0,1132.376003,1224.156868,33.650795,34.987953,33.650795,34.987953,"['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P189;"", [""VAR_URI_VA..."


In [46]:
1+1

2

### Archivos enteros

In [47]:
import os
path = "/home/school/math/final_analysis.csv"

In [48]:
os.remove(URL + 'rl_csvs/ql_df.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/ccarmona/memoria/dataset/rl_csvs/ql_df.csv'

In [ ]:
os.remove(URL + 'rl_csvs/sarsa_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/neo_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/exsarsa_df.csv')

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/sarsa_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/exsarsa_df.csv',index=False)